# Introduction
The Domain Class is used to handle signals, or ranges of values sorted from min-max which will have some unit attatched. Most of the time users of this package will not directly interact with the Domain Class, but this notebook should provide the necessary understanding.

In [4]:
from dreye import Domain
from dreye.err import DreyeError, DreyeUnitError
from dreye.constants import ureg, ABSOLUTE_ACCURACY

First, intialize your domain using one of two ways:
 - Define the start value, the end value the step size, and the units 
 - Create a list of unique values, and attatch the unit to it. This method is similar to a single dimensional numpy array

First, intialize your domain using one of two ways:
 - Define the start value, the end value the step size, and the units 
 - Create a list of unique values, and attatch the unit to it. This method is similar to a single dimensional numpy array

In [2]:
domain = Domain(-0.5, 1, 0.1, 's')
domain2 = Domain([-0.5, 0, 0.5, 1.1], units='s')
# will always be ascending

In [5]:
try:
    domain.units = 'V'
except DreyeUnitError as e:
    print(e)

Cannot convert from 'V' ([length] ** 2 * [mass] / [current] / [time] ** 3) to 'second' ([time]) for instance of type Domain.


In [7]:
ureg(str(domain.units))

1 <Unit('second')>

You can always check if the domain you created is uniform. 

Domain has many parameters and attributes- review the documentation for more details. Examples are shown below.

In [5]:
print(domain)
print(domain.is_uniform)
print(domain.start)
print(domain.end)
print(domain.interval)
print(domain.span)
print(domain.gradient)
print(domain.units)
print(domain.magnitude)
print(domain.values)

Domain(start=-0.5, end=1.0, interval=0.1, units=second, dtype=<class 'numpy.float64'>)
True
-0.5
1.0
0.1
1.5
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1] second
second
[-0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8
  0.9  1. ]
[-0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ] second


You can convert units (for example, from s to ms). UREG, or unit registration, will check if the conversion is compatible and complete it if possible. 

You can muliply domain by a number or by units. You can also add to the array to shift it, but the units have to match. 

In [6]:
print(domain.to('ms'))
print(domain * 3 * ureg('s'))
print(domain + 3 * ureg('s'))

Domain(start=-500.0, end=1000.0, interval=100.0, units=millisecond, dtype=<class 'numpy.float64'>)
Domain(start=-1.5, end=3.0, interval=0.3, units=second ** 2, dtype=<class 'numpy.float64'>)
Domain(start=2.5, end=4.0, interval=0.1, units=second, dtype=<class 'numpy.float64'>)


If your domain is non-uniform, you can enforce uniformity. This is particularly useful for interpolation purposes. **enforce_uniformity** returns new domain where the interval are uniform and the interval length is the average interval calulcated from the original domain. 

**equalize_domains** is useful when aligning or averaging Domains. Domains need to be uniform before this will work. After enforcing uniformity if necessary, **equalize_domains** will chose the most common denominator for what the domain range should be. Specifically, it will take the largest **Start** value, and smallest **End** value. The interval will be the largest interval represented in the two original domains. 

As a broad concept, you can compare this to x values being used for interpolation of y values.

If your domain is non-uniform, you can enforce uniformity. This is particularly useful for interpolation purposes. **enforce_uniformity** returns new domain where the interval are uniform and the interval length is the average interval calulcated from the original domain. 

**equalize_domains** is useful when aligning or averaging Domains. Domains need to be uniform before this will work. After enforcing uniformity if necessary, **equalize_domains** will chose the most common denominator for what the domain range should be. Specifically, it will take the largest **Start** value, and smallest **End** value. The interval will be the largest interval represented in the two original domains. 

As a broad concept, you can compare this to x values being used for interpolation of y values.

In [31]:
other_domain = Domain([0.1, 0.3, 0.4, 0.6, 0.8], units='s')
print(other_domain)
print(other_domain.interval)
print(other_domain.is_uniform)
print(other_domain.enforce_uniformity())
try:
    domain.equalize_domains(other_domain)
except DreyeError as e:
    print(e)
print(domain.equalize_domains(other_domain.enforce_uniformity()))
try:
    other_domain.start = 0
except DreyeError as e:
    print(e)

Domain(start=0.1, end=0.8, interval=[0.2 0.1 0.2 0.2], units=second)
[0.2 0.1 0.2 0.2]
False
Domain(start=0.1, end=0.8, interval=0.17500000000000002, units=second)
Domains must be uniform for operations on signal classes, if domains are not equal.
Domain(start=0.1, end=0.8, interval=0.17500000000000002, units=second)
start attribute: 0 value below previous start value 0.1


In [5]:
other_domain = Domain(1, 2, 0.1, 's')
print(other_domain == domain)
other_domain = Domain(0, 1, 0.1, 'V')
print(other_domain == domain)
other_domain = Domain(0, 2, 0.2, 's')
print(other_domain == domain)
new_domain = other_domain.equalize_domains(domain)
print(new_domain)

False
False
False
Domain(start=0.0, end=1.0, interval=0.2, units=second, dtype=<class 'numpy.float64'>)


In [6]:
domain.save("data/domain_example.json")

In [7]:
domain_loaded = Domain.load("data/domain_example.json")

In [8]:
domain_loaded == domain

True